In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")


Success - the MySageMakerInstance is in the us-east-1 region. You will use the 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [2]:
bucket_name = 'sagemakeruniversityprediction' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [9]:
# try:
#   urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
#   print('Success: downloaded bank_clean.csv.')
# except Exception as e:
#   print('Data load error: ',e)

try:
  model_data = pd.read_csv('./university_train_data.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


In [10]:
print(model_data.head())

   admit  gre   gpa  rank
0      0  380  3.61     3
1      1  660  3.67     3
2      1  800  4.00     1
3      1  640  3.19     4
4      0  520  2.93     4


In [11]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(280, 4) (120, 4)


In [13]:
train_data.to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [14]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[my_region],role, train_instance_count=1, train_instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)


In [15]:
xgb.fit({'train': s3_input_train})

2019-12-24 01:48:14 Starting - Starting the training job...
2019-12-24 01:48:15 Starting - Launching requested ML instances......
2019-12-24 01:49:17 Starting - Preparing the instances for training......
2019-12-24 01:50:37 Downloading - Downloading input data
2019-12-24 01:50:37 Training - Downloading the training image...
2019-12-24 01:51:08 Uploading - Uploading generated training model
2019-12-24 01:51:08 Completed - Training job completed
Arguments: train
[2019-12-24:01:50:56:INFO] Running standalone xgboost training.
[2019-12-24:01:50:56:INFO] Path /opt/ml/input/data/validation does not exist!
[2019-12-24:01:50:56:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8515.82mb
[2019-12-24:01:50:56:INFO] Determined delimiter of CSV input is ','
[01:50:56] S3DistributionType set as FullyReplicated
[01:50:56] 280x3 matrix with 840 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[01:50:56] src/tree/updater_prune

In [16]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

--------------------------------------------------------------------------------------------------------------!

In [26]:
groundtruth = test_data['admit'].values
test_data_array = test_data.drop(['admit'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
for i in range(len(predictions_array)):
    if predictions_array[i]>=0.5:
        predictions_array[i] = 1
    else:
        predictions_array[i] = 0
        
print("test accuracy of the dataset is", (groundtruth == predictions_array).mean())

test accuracy of the dataset is 0.7333333333333333


In [30]:
cm = pd.crosstab(index=test_data['admit'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "Reject", "Admit"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("Reject", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Admit", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))



Overall Classification Rate: 73.3%

Predicted      Reject            Admit
Observed
Reject         77% (80)    50% (8)
Admit           23% (24)     50% (8) 

